In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sklearn
from sklearn import tree
from sklearn.model_selection import train_test_split

from scipy import ndimage
from skimage.io import imread
from skimage import color
from skimage.transform import resize
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import AxesGrid
import time
import cv2
from PIL import Image
%matplotlib inline
import scipy.misc
from sklearn.svm import SVC

from imblearn import under_sampling, over_sampling


from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier, AdaBoostClassifier
from sklearn import preprocessing

In [2]:
def BorderPixels_in(mask):
    (n,p) = np.shape(mask)
    
    border = []
    for i in range(n):
        for j in range (p):
            
            if (mask[i,j]>0.7 and (mask[max(i-1,0),j]<0.1 or mask[min(i+1,n-1),j]<0.1 or mask[i,max(j-1,0)]<0.1 or mask[i,min(j+1,p-1)]<0.1)) :
                border.append((i,j))
    return(border)

In [37]:
image = imread('data/im_resized/IM_000000_resized.jpg')
image_S = imread('data/im_resized/IM_000000_Segmentation_resized.jpg')
image_S = image_S.astype(bool)
        
#get the border
border =BorderPixels_in(image_S)

#On convertit l'image en niveau de gris
gray_image = color.rgb2gray(image)*255
n,p = gray_image.shape
R = []
for coord in border :
    (i,j)=coord
            
    x_bar = 0
    for l in range (max(0,i-7),min(i+7,n)):
        for k in range (max(0,j-7),min(j+7,p)):
            x_bar += gray_image[i,j]
    x_bar = x_bar/(len(border))
    
    SSE = 0
    SST = 0
    for l in range (max(0,i-7),min(i+7,n)):
        for k in range (max(0,j-7),min(j+7,p)):
            
            if image_S[l,k] :
                SST +=  (gray_image[i,j]-x_bar)**2
                
            else :
                SSE +=  (gray_image[i,j]-x_bar)**2
    R.append(SSE/SST)
R = np.asarray(R)
f1 = np.percentile(R,25)
f2 = np.percentile(R,50)                
f3 = np.percentile(R,75)


In [26]:
f3

1.5128205128205128

In [3]:
feat_df = pd.read_csv('data/features.csv', sep=',')


def features_border(idRep):
    t = time.time()
    m=0
    for x in idRep:
        m+=1
        filename = 'data/im_resized/{}_resized.jpg'.format(x)
        filename_S = 'data/im_resized/{}_Segmentation_resized.jpg'.format(x)
        image = imread(filename)
        image_S = imread(filename_S)
        image_S = image_S.astype(bool)
        
        #get the border
        border =BorderPixels_in(image_S)

        #On convertit l'image en niveau de gris
        gray_image = color.rgb2gray(image)*255
        n,p = gray_image.shape
        R = []
        for coord in border :
            (i,j)=coord
            
            x_bar = 0
            for l in range (max(0,i-7),min(i+7,n)):
                for k in range (max(0,j-7),min(j+7,p)):
                    x_bar += gray_image[i,j]
            x_bar = x_bar/(len(border))
    
            SSE = 0
            SST = 0
            for l in range (max(0,i-7),min(i+7,n)):
                for k in range (max(0,j-7),min(j+7,p)):
            
                    if image_S[l,k] :
                        SST +=  (gray_image[i,j]-x_bar)**2
                
                    else :
                        SSE +=  (gray_image[i,j]-x_bar)**2
            R.append(SSE/SST)
        R = np.asarray(R)
        f1 = np.percentile(R,25)
        f2 = np.percentile(R,50)                
        f3 = np.percentile(R,75)
        
        
        if(m%50 ==0):
            print(time.time()-t)
            print(m," : tour 1")
        feat_df.loc[feat_df['ImageId'] == x, 'border1'] = f1
        feat_df.loc[feat_df['ImageId'] == x, 'border2'] = f2
        feat_df.loc[feat_df['ImageId'] == x, 'border3'] = f3
    print(time.time()-t)
    #feat_df.to_csv('data/features.csv', index=None, sep=',', mode='w')
feat_df  = feat_df[feat_df['ImageId']!='IM_000720']
#features_border(feat_df['ImageId'].values)

In [39]:
feat_df.to_csv('data/features.csv', index=None, sep=',', mode='w')

In [32]:
features_border(feat_df['ImageId'].values)

36.07957577705383
50  : tour 1


/Users/raphael/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:41: RuntimeWarning: invalid value encountered in double_scalars
/Users/raphael/anaconda3/lib/python3.6/site-packages/numpy/lib/function_base.py:4274: RuntimeWarning: Invalid value encountered in percentile
  interpolation=interpolation)


75.29481887817383
100  : tour 1
111.26355409622192
150  : tour 1
150.88287806510925
200  : tour 1
187.00806212425232
250  : tour 1
222.43746995925903
300  : tour 1
260.72883200645447
350  : tour 1
301.0398631095886
400  : tour 1
346.31601905822754
450  : tour 1
385.8543450832367
500  : tour 1
419.04185700416565
550  : tour 1
448.27450299263
600  : tour 1
481.57619404792786
650  : tour 1
512.8889360427856
700  : tour 1
542.3101530075073
750  : tour 1
577.3354389667511
800  : tour 1
623.7117309570312
850  : tour 1
658.1650369167328


In [4]:
def features_border2(idRep):
    t = time.time()
    m=0
    for x in idRep:
        m+=1
        filename = 'data/im_resized/{}_resized.jpg'.format(x)
        filename_S = 'data/im_resized/{}_Segmentation_resized.jpg'.format(x)
        image = imread(filename)
        image_S = imread(filename_S)
        image_S = image_S.astype(bool)
        
        #get the border
        border =BorderPixels_in(image_S)

        #On convertit l'image en niveau de gris
        gray_image = color.rgb2gray(image)*255
        n,p = gray_image.shape
        R = []
        for coord in border :
            (i,j)=coord
            
            x_bar = 0
            for l in range (max(0,i-7),min(i+7,n)):
                for k in range (max(0,j-7),min(j+7,p)):
                    x_bar += gray_image[i,j]
            x_bar = x_bar/(len(border))
    
            SSE = 0
            SST = 0
            for l in range (max(0,i-7),min(i+7,n)):
                for k in range (max(0,j-7),min(j+7,p)):
            
                    if not(image_S[l,k]) :
                        SSE +=  (gray_image[i,j]-x_bar)**2
                    SST +=  (gray_image[i,j]-x_bar)**2
            R.append(SSE/SST)
        R = np.asarray(R)
        f1 = np.percentile(R,25)
        f2 = np.percentile(R,50)                
        f3 = np.percentile(R,75)
        
        
        if(m%50 ==0):
            print(time.time()-t)
            print(m," : tour 1")
        feat_df.loc[feat_df['ImageId'] == x, 'border1NEW'] = f1
        feat_df.loc[feat_df['ImageId'] == x, 'border2NEW'] = f2
        feat_df.loc[feat_df['ImageId'] == x, 'border3NEW'] = f3
    print(time.time()-t)
    #feat_df.to_csv('data/features.csv', index=None, sep=',', mode='w')
feat_df  = feat_df[feat_df['ImageId']!='IM_000720']
features_border2(feat_df['ImageId'].values)

29.293370962142944
50  : tour 1


/Users/raphael/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:36: RuntimeWarning: invalid value encountered in double_scalars
/Users/raphael/anaconda3/lib/python3.6/site-packages/numpy/lib/function_base.py:4274: RuntimeWarning: Invalid value encountered in percentile
  interpolation=interpolation)


63.67914295196533
100  : tour 1
95.70532202720642
150  : tour 1
128.85044693946838
200  : tour 1
159.74652075767517
250  : tour 1
190.48953986167908
300  : tour 1
223.61757707595825
350  : tour 1
255.75494003295898
400  : tour 1
289.38665986061096
450  : tour 1
323.80470085144043
500  : tour 1
356.6043426990509
550  : tour 1
385.8032898902893
600  : tour 1
419.76479482650757
650  : tour 1
450.72752594947815
700  : tour 1
480.5172309875488
750  : tour 1
512.2036678791046
800  : tour 1
543.6049847602844
850  : tour 1
578.230222940445


In [5]:
feat_df

,ImageId,ImageSegId,Int/Out A,Int/Out B,Int/Out L,Mean Red,Mean Green,Mean Blue,Std Red,Std Green,...,f1,f2,color_Asm1,color_Asm2,border1,border2,border3,border1NEW,border2NEW,border3NEW
0,IM_000839,IM_000839_Segmentation,6.592233,-5.344395,-24.464932,141.403305,76.138858,59.356348,33.051904,35.996311,...,15.431855,15.278272,6.623157,0.102065,0.568000,0.884615,1.545455,0.362245,0.469388,0.607143
1,IM_000764,IM_000764_Segmentation,9.435975,11.052303,-17.513378,177.934414,140.767870,119.267686,21.566450,33.102061,...,19.792004,18.343589,2.841392,0.067162,0.633333,0.940594,1.481013,0.387755,0.484694,0.596939
2,IM_000847,IM_000847_Segmentation,12.067892,12.272357,-26.932157,191.121266,146.230489,126.174010,38.183883,51.732835,...,34.858943,20.127427,9.573378,0.331451,0.619835,0.921569,1.450000,0.382653,0.479592,0.591837
3,IM_000503,IM_000503_Segmentation,-2.991938,-3.219282,-47.724101,98.635038,86.355673,81.401641,69.890602,65.428576,...,27.045493,23.562528,3.763290,0.032292,0.593496,0.875641,1.390244,0.372449,0.466837,0.581633
4,IM_000169,IM_000169_Segmentation,-1.834943,-5.427625,-29.019240,132.197961,101.061412,107.357242,57.599711,52.337378,...,30.616099,33.460547,30.767867,2.396294,0.580645,0.884615,1.481013,0.367347,0.469388,0.596939
5,IM_000402,IM_000402_Segmentation,3.882130,8.448086,-5.264763,183.925657,145.846607,120.390153,4.923161,11.711033,...,17.635936,16.516673,7.142128,0.997428,0.633333,1.000000,1.545455,0.387755,0.500000,0.607143
6,IM_000234,IM_000234_Segmentation,-2.037647,-5.062983,-8.463933,143.491822,127.863590,122.154165,29.189866,34.684501,...,21.825817,20.138314,7.627865,0.313743,0.258427,0.543307,0.979798,0.205357,0.352041,0.494898
7,IM_000474,IM_000474_Segmentation,11.598973,-0.759587,-28.975201,182.277040,113.185881,83.681671,39.114360,47.640542,...,29.550356,38.563957,5.615173,0.115184,0.568000,0.902913,1.450000,0.362245,0.474490,0.591837
8,IM_000143,IM_000143_Segmentation,1.234025,-5.308873,-26.251819,163.460245,105.634082,85.365743,43.773489,39.777484,...,18.803481,18.925384,7.403649,0.135362,0.555556,0.884615,1.481013,0.357143,0.469388,0.596939
9,IM_000134,IM_000134_Segmentation,8.298158,4.062482,-27.984594,142.853286,112.818936,108.372382,45.039484,53.827806,...,32.089934,22.228857,37.837421,4.153339,0.593496,0.866667,1.450000,0.372449,0.464286,0.591837


In [6]:
for x in (feat_df['ImageId'].values):
    if np.isnan(np.min(feat_df.loc[feat_df['ImageId']==x, 'border1NEW'].values)):
        feat_df.loc[feat_df['ImageId'] == x, 'border1NEW'] = 0.35
        feat_df.loc[feat_df['ImageId'] == x, 'border2NEW'] = 0.47
        
        feat_df.loc[feat_df['ImageId'] == x, 'border3NEW'] = 0.59
    

In [7]:
np.min(feat_df.loc[feat_df['ImageId']=='IM_000000', 'border1NEW'].values)

0.39285714285714346

In [8]:
np.isnan(np.min(feat_df['border1NEW'].values))

False

In [9]:
feat_df.to_csv('data/features.csv', index=None, sep=',', mode='w')